In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk.corpus.reader.wordnet import NOUN, ADJ, ADV, VERB

In [2]:
# Téléchargement des ressources nécessaires
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('sentiwordnet')

[nltk_data] Downloading package punkt to C:\Users\user.user-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user.user-PC\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to C:\Users\user.user-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to C:\Users\user.user-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [3]:
# Dictionnaire pour la correspondance entre les POS tags de NLTK et ceux de WordNet
POS_TAG_MAP = {
    'N': 'n',  # Noun
    'V': 'v',  # Verb
    'R': 'r',  # Adverb
    'J': 'a'   # Adjective
}

In [4]:
# Convertir les POS tags de NLTK à ceux utilisés par WordNet
def get_wordnet_pos(pos_tag):
    return POS_TAG_MAP.get(pos_tag[0], None)

In [6]:
# Analyse de sentiment d'une phrase sans gestion de la négation
def analyze_sentiment(sentence):
    tokens = word_tokenize(sentence)
    print("Tokens:", tokens)

    pos_tags = pos_tag(tokens)
    print("POS Tags:", pos_tags)

    wnl = WordNetLemmatizer()
    lemmas = [wnl.lemmatize(token) for token in tokens]
    print("Lemmas:", lemmas)

    total_score = 0
    for token, pos in zip(lemmas, pos_tags):
        wn_pos = get_wordnet_pos(pos[1])
        if wn_pos:
            senti_synsets = list(swn.senti_synsets(token, wn_pos))
            score=0
            if senti_synsets:
                score = senti_synsets[0].pos_score() - senti_synsets[0].neg_score()
                print("Token: {}, Positivity: {}, Negativity: {}, Objectivity: {}, Score: {}".format(token, senti_synsets[0].pos_score(), senti_synsets[0].neg_score(), senti_synsets[0].obj_score(), score))

            total_score += score
    print("Total Sentiment Score:", total_score)


In [8]:
# Analyse de sentiment d'une phrase avec gestion de la négation

# Méthode 1: inverser la polarité du mot qui vient juste après la négation
def analyze_sentiment_with_negation1(sentence):
    tokens = word_tokenize(sentence)
    print("Tokens:", tokens)

    pos_tags = pos_tag(tokens)
    print("POS Tags:", pos_tags)

    wnl = WordNetLemmatizer()
    lemmas = [wnl.lemmatize(token) for token in tokens]
    print("Lemmas:", lemmas)

    total_score = 0
    negative_words = ["not", "never", "no", "n't", "hardly", "scarcely", "barely"]
    negate = False
    for token, pos in zip(lemmas, pos_tags):
        wn_pos = get_wordnet_pos(pos[1])
        if wn_pos:
            senti_synsets = list(swn.senti_synsets(token, wn_pos))
            if senti_synsets:
                score = senti_synsets[0].pos_score() - senti_synsets[0].neg_score()
                if negate:
                    score = -score
                total_score += score
                print("Token: {}, Positivity: {}, Negativity: {}, Objectivity: {}, Score: {}".format(token, senti_synsets[0].pos_score(), senti_synsets[0].neg_score(), senti_synsets[0].obj_score(), score))


        # Gestion de la négation (simple)
        if token.lower() in negative_words:
            negate = True
        else:
            negate = False

    print("Total Sentiment Score:", total_score)



In [10]:
# Méthode 2: inverser la poolarité de tous les mots qui viennent après la négation
def analyze_sentiment_with_negation2(sentence):
    tokens = word_tokenize(sentence)
    print("Tokens:", tokens)

    pos_tags = pos_tag(tokens)
    print("POS Tags:", pos_tags)

    wnl = WordNetLemmatizer()
    lemmas = [wnl.lemmatize(token) for token in tokens]
    print("Lemmas:", lemmas)

    total_score = 0
    negate = False
    negative_words = ["not", "never", "no", "n't", "hardly", "scarcely", "barely"]

    for token, pos in zip(lemmas, pos_tags):
        wn_pos = get_wordnet_pos(pos[1])
        if wn_pos:
            senti_synsets = list(swn.senti_synsets(token, wn_pos))
            if senti_synsets:
                score = senti_synsets[0].pos_score() - senti_synsets[0].neg_score()
                if negate:
                    score = -score
                total_score += score
                print("Token: {}, Positivity: {}, Negativity: {}, Objectivity: {}, Score: {}".format(token, senti_synsets[0].pos_score(), senti_synsets[0].neg_score(), senti_synsets[0].obj_score(), score))

        # Gestion de la négation
        if token.lower() in negative_words:
            negate = not negate

    print("Total Sentiment Score:", total_score)


In [11]:
# Analyse de sentiment pour les deux phrases sans gestion de la négation
print("\nAnalyse de sentiment sans négation pour 'I love this beautiful day.':")
analyze_sentiment("I love this beautiful day.")

print("\nAnalyse de sentiment avec négation pour 'I do not love this beautiful day.':")
analyze_sentiment("I do not love this beautiful day.")

# Analyse de sentiment pour la phrase avec gestion de la négation Méthode 1
print("\nMéthode 1: Analyse de sentiment avec négation pour 'I do not love this beautiful day.':")
analyze_sentiment_with_negation1("I do not love this beautiful day.")

# Analyse de sentiment pour la phrase avec gestion de la négation Méthode 2
print("\nMéthode 2: Analyse de sentiment avec négation pour 'I do not love this beautiful day.':")
analyze_sentiment_with_negation2("I do not love this beautiful day.")



Analyse de sentiment sans négation pour 'I love this beautiful day.':
('Tokens:', ['I', 'love', 'this', 'beautiful', 'day', '.'])
('POS Tags:', [('I', 'PRP'), ('love', 'VBP'), ('this', 'DT'), ('beautiful', 'JJ'), ('day', 'NN'), ('.', '.')])
('Lemmas:', ['I', 'love', 'this', 'beautiful', 'day', '.'])
Token: love, Positivity: 0.5, Negativity: 0.0, Objectivity: 0.5, Score: 0.5
Token: beautiful, Positivity: 0.75, Negativity: 0.0, Objectivity: 0.25, Score: 0.75
Token: day, Positivity: 0.0, Negativity: 0.0, Objectivity: 1.0, Score: 0.0
('Total Sentiment Score:', 1.25)

Analyse de sentiment avec négation pour 'I do not love this beautiful day.':
('Tokens:', ['I', 'do', 'not', 'love', 'this', 'beautiful', 'day', '.'])
('POS Tags:', [('I', 'PRP'), ('do', 'VBP'), ('not', 'RB'), ('love', 'VB'), ('this', 'DT'), ('beautiful', 'JJ'), ('day', 'NN'), ('.', '.')])
('Lemmas:', ['I', 'do', 'not', 'love', 'this', 'beautiful', 'day', '.'])
Token: do, Positivity: 0.0, Negativity: 0.0, Objectivity: 1.0, Sco